### Workbook for extracting WorldPop population estimates for Uganda bridge catchment areas
Week of August 4, 2025
<br>
Author: Adele Birkenes

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import rasterio
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium import GeoJson

**Task 1: Read in data on bridge sites and village boundaries**

Note: The village boundaries are not used in this analysis, but I am checking whether they contain a custom Uganda projection

In [6]:
synced_path = "../../synced-data/population-exploration/Uganda"
unsynced_path = "../../unsynced-data"

# Village boundaries (geodataframe)
village_boundaries_fp = os.path.join(synced_path,"uga_admbnda_ubos_20200824_shp/uga_admbnda_adm4_ubos_20200824.shp")
Uganda_village_boundaries = gpd.read_file(village_boundaries_fp)
print("Uganda village boundaries CRS:", Uganda_village_boundaries.crs)

# Bridge sites (dataframe)
bridge_sites_fp = os.path.join(synced_path, "All B2P Uganda Sites_2025.07.14.csv")
Uganda_bridge_sites = pd.read_csv(bridge_sites_fp)

Uganda village boundaries CRS: EPSG:4326


Conclusion: The village boundaries are unprojected (WGS84). However, it is appropriate to use a projected CRS for distance calculations such as buffers. Therefore, I will use UTM zone 36N (EPSG:32636), which covers most of the country.

**Task 2: Check for rows with missing/invalid coordinates, which can create invalid geometries and cause problems. Create a dataframe of invalid bridge sites for review**

In [25]:
# Check for missing or obviously invalid coordinates
print(Uganda_bridge_sites[["GPS (Latitude)", "GPS (Longitude)"]].isnull().sum())
print(Uganda_bridge_sites[["GPS (Latitude)", "GPS (Longitude)"]].describe())
print(bridge_points.is_valid.value_counts())

GPS (Latitude)     12
GPS (Longitude)    12
dtype: int64
       GPS (Latitude)  GPS (Longitude)
count      598.000000       598.000000
mean         0.641824        31.522621
std          0.913415         1.925981
min         -1.459017        29.630709
25%          0.098429        30.071547
50%          0.674117        30.303297
75%          1.064221        34.289442
max          3.750561        35.355083
True     598
False     12
Name: count, dtype: int64


In [27]:
# Create a dataframe of rows with missing latitude or longitude
Uganda_bridge_sites_missing_coords = Uganda_bridge_sites[
    Uganda_bridge_sites["GPS (Latitude)"].isnull() | Uganda_bridge_sites["GPS (Longitude)"].isnull()
]

# Export to CSV
Uganda_bridge_sites_missing_coords_fp = os.path.join(synced_path, "Uganda_bridge_sites_missing_coords.csv")
Uganda_bridge_sites_missing_coords.to_csv(Uganda_bridge_sites_missing_coords_fp, index=False)

**Task 3: Create a geodataframe of bridge sites with valid coordinates; use custom Uganda projection**

In [33]:
# Convert bridge sites dataframe to geodataframe that has custom Uganda projection
def map_bridges(bridges, bridges_lat, bridges_lon, projection='EPSG:32636'):

    # Remove rows with missing lat/lon values
    bridges = bridges.dropna(subset=[bridges_lat, bridges_lon])

    # Create lat/lon variables from filtered dataframe
    lon = bridges[bridges_lon]
    lat = bridges[bridges_lat]

    # Create gdf of bridges data by converting lat/lon values to list of Shapely Point objects
    bridge_points = gpd.GeoDataFrame(bridges, geometry=gpd.points_from_xy(x=lon, y=lat), crs='EPSG:4326')

    # Set CRS of bridges gdf to custom Uganda projection
    bridge_points = bridge_points.to_crs(projection)

    # Check that reprojection was successful
    print(f'CRS of bridge sites: {bridge_points.crs}')
    
    return bridge_points

bridge_points = map_bridges(bridges = Uganda_bridge_sites,
                            bridges_lat = "GPS (Latitude)",
                            bridges_lon = "GPS (Longitude)")

#bridge_points.explore()

CRS of bridge sites: EPSG:32636


**Task 4: Create 3 km buffers around bridge sites**

In [34]:
def create_bridge_buffers(bridges, buffer_distance):

    # Create buffers around bridges according to user's distance input
    buf = bridges.geometry.buffer(distance=buffer_distance)

    # Add bridges point attribute information to buffers
    bridge_buffers = bridges.copy()
    bridge_buffers['geometry'] = buf

    return bridge_buffers

bridge_buffers_3km = create_bridge_buffers(bridges = bridge_points, buffer_distance = 3000)

#bridge_buffers_3km.explore()